<a href="https://colab.research.google.com/github/dmscornjs/Hotelreview/blob/main/%EC%95%BC%EB%86%80%EC%9E%90%ED%81%AC%EB%A1%A4%EB%A7%81%EC%A3%BC%EC%84%9D%EC%A0%95%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 야놀자 크롤링 제출용 정리

본 코드는 코랩 환경 작성되었습니다.

# 코랩에서 selenium을 사용하기 위해 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

# 필요한 패키지 불러오기

In [ ]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta

 야놀자 사이트에서 호텔로 구분되는 것 중 지역 선택에서 서울 전체로 구분되는 호텔들을 모아 그 리뷰를 전부 수집하였다.

# 호텔별 URL을 가져오기

In [ ]:
# url 함수
def make_url(Url):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
  driver.get(Url)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')

  # 무한 스크롤
  import time
  time.sleep(1)

  SCROLL_PAUSE_TIME = 1.5

  # Get scroll height
  last_height = driver.execute_script("return document.body.scrollHeight")

  while True:
      # Scroll down to bottom
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

      # Wait to load page
      time.sleep(SCROLL_PAUSE_TIME)
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
      time.sleep(SCROLL_PAUSE_TIME)

      # Calculate new scroll height and compare with last scroll height
      new_height = driver.execute_script("return document.body.scrollHeight")

      if new_height == last_height:
          break

      last_height = new_height  

  data = soup.select("section > div > div > div > a")

  for d in data:
    hotelurl.append(d.attrs['href'])
  driver.close()
  

In [ ]:
hotelurl = []
make_url("https://www.yanolja.com/hotel/r-900582?advert=AREA&topAdvertiseMore=1")

# 리뷰 수집

In [ ]:
def yanolja_crawl(url):
    global df # 전역 변수를 지역 범위에서 적용
    #Url = "https://www.yanolja.com"+url
    driver.get(url)


    hotel = driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf h1').text
    hotel = hotel.replace("[★숙박대전] ", "")

    # 리뷰 페이지 클릭 (Click review page)
    driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf > a').click()

    # 무한 스크롤
    import time
    time.sleep(1)

    SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        last_height = new_height

    # 스크롤 끝나고 페이지 elements 가져오기
    raw = driver.page_source # 페이지의 elements 모두 가져오기
    html = BeautifulSoup(raw, "html.parser")

    # 리뷰 텍스트 선택자: div.item-wrap
    reviews = html.select("div.item-wrap")

    columns = ['hotel', 'star', 'review', 'date']

    for r in reviews:
        review = r.select_one("p")
        date = r.select_one('time')

        stars = r.select('.container.score')
        for s in stars:
            star_count = 5 - len(s.select(".ico.gray"))

        df2 = df2.append(pd.DataFrame([[hotel, star_count, review.text, date.text]], columns = columns),ignore_index=True)
    
def change_date(x):
    now = datetime.datetime.today()
    today = now.strftime("%Y. %m. %d")
    yesterday = (now - timedelta(days=1)).strftime("%Y. %m. %d")
    _2days_ago = (now - timedelta(days = 2)).strftime("%Y. %m. %d")
    _3days_ago = (now - timedelta(days = 3)).strftime("%Y. %m. %d")
    hour = int(datetime.datetime.today().strftime("%H"))
    if "시간 전" in x:
        x = int(x.replace("시간 전", ""))
        if hour - x > 0: return today
        else: return yesterday
    elif "일 전" in x:
        if "1" in x: return yesterday
        elif "2" in x: return _2days_ago
        else: return _3days_ago
    else: return x

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
for url in hotelurl:
    Url = "https://www.yanolja.com"+url
    yanolja_crawl(Url)

driver.close()

In [ ]:
#날짜 형식 변환
df['real_date'] = df['date'].apply(change_date)

In [ ]:
df.describe()

In [ ]:
df.to_csv("야놀자호텔리뷰.csv")